In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text("container","raw")
dbutils.widgets.text("catalogo","credit_catalog")
dbutils.widgets.text("esquema","bronze")
dbutils.widgets.text("datalake","adlssmartdata1202")

In [0]:
container = dbutils.widgets.get("container")
catalogo = dbutils.widgets.get("catalogo")
esquema = dbutils.widgets.get("esquema")
datalake = dbutils.widgets.get("datalake")

ruta = f"abfss://{container}@{datalake}.dfs.core.windows.net/credit_record.csv"

In [0]:
df_credit_record = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(ruta)

In [0]:
credit_record_schema = StructType(fields=[StructField("ID", IntegerType(), False),
                                     StructField("MONTHS_BALANCE", IntegerType(), True),
                                     StructField("STATUS", StringType(), True)
])

In [0]:
df_credit_record_final = spark.read\
.option('header', True)\
.schema(credit_record_schema)\
.csv(ruta)

In [0]:
df_credit_record_final_date = df_credit_record_final.withColumn("INGESTION_DATE", current_timestamp())

In [0]:
df_credit_record_final_date.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.credit_record")